#Test 2.1 Jacobian Code

In [1]:
import eoldas_ng
from eoldas_ng import FIXED, VARIABLE, CONSTANT
from eoldas_ng import eoldas_observation_helpers
from eoldas_ng import eoldas_helpers
import gp_emulator

import numpy as np
from collections import OrderedDict
#from eoldas_ng import Prior, SpatialSmoother
#from eoldas_ng import ObservationOperatorImageGP, StandardStatePROSAIL

Set up the true state. A 3 by 4 grid with one observation at 1,2 and the remaining area masked out.

In [2]:
state_grid = np.zeros((3,4))

#obslocation = (slice(1,3),slice(2,4))
obslocation = (1,2)
examplelai = 3
lai_true=np.zeros_like(state_grid)
mask= np.zeros_like(state_grid, dtype=bool)
lai_true[obslocation]=examplelai
mask[obslocation]=True

print 'True lai= ', lai_true
print 'mask = ', mask

True lai=  [[ 0.  0.  0.  0.]
 [ 0.  0.  3.  0.]
 [ 0.  0.  0.  0.]]
mask =  [[False False False False]
 [False False  True False]
 [False False False False]]


In [3]:
# Create the state configuration dictionary that states which parameters are fixed or variable. Then create
# The state operator: the_state. THis is set up to use PROSAIL

state_config = OrderedDict ()
state_config['n'] = FIXED
state_config['cab'] = FIXED
state_config['car'] = FIXED
state_config['cbrown'] = FIXED
state_config['cw'] = FIXED
state_config['cm'] = FIXED
state_config['lai'] = VARIABLE
state_config['ala'] = FIXED
state_config['bsoil'] = FIXED
state_config['psoil'] = FIXED

                                  
optimisation_options={"factr": 1e12,
                "m":400, "pgtol":1e-4, "maxcor":200,
                "maxiter":1500, "disp":True}

the_state = eoldas_helpers.StandardStatePROSAIL(state_config, state_grid,
                 optimisation_options=optimisation_options,
                 output_name="JacobianTest", verbose=True)


Saving results to JacobianTest.pkl


In [4]:
# reset the default to match our example lai and save a copy of the true state.
the_state.default_values['lai'] = examplelai
state_true = the_state.default_values.copy()
state_true['lai'] = lai_true

# Create a copy of state where our lai is shifted by an amout epsilon for finite difference testing.
epsilon = 0.000001
delta_lai = lai_true.copy()
delta_lai[obslocation] += epsilon
delta_state = state_true.copy()
delta_state['lai'] = delta_lai

# Confirm our states are as expected 
print 'True state = ', state_true
print 'Shifted state = ',delta_state
print 'default values = ', the_state.default_values


True state =  OrderedDict([('n', 1.6), ('cab', 20.0), ('car', 1.0), ('cbrown', 0.01), ('cw', 0.018), ('cm', 0.03), ('lai', array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  3.,  0.],
       [ 0.,  0.,  0.,  0.]])), ('ala', 70.0), ('bsoil', 0.5), ('psoil', 0.9)])
Shifted state =  OrderedDict([('n', 1.6), ('cab', 20.0), ('car', 1.0), ('cbrown', 0.01), ('cw', 0.018), ('cm', 0.03), ('lai', array([[ 0.      ,  0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  3.000001,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ]])), ('ala', 70.0), ('bsoil', 0.5), ('psoil', 0.9)])
default values =  OrderedDict([('n', 1.6), ('cab', 20.0), ('car', 1.0), ('cbrown', 0.01), ('cw', 0.018), ('cm', 0.03), ('lai', 3), ('ala', 70.0), ('bsoil', 0.5), ('psoil', 0.9)])


In [5]:
# We will be calling the jacobian and cost calulation directly from the observation operator.
# This expects the state to be in transformed space so we calculate the transformed values
# for the original and shifted state

# Turn the state dictionary into a vector of the transformed variables.
trans_delta_state_vec = the_state.pack_from_dict(delta_state, do_transform=True)
trans_state_vec = the_state.pack_from_dict(state_true, do_transform=True)
#put the transformed vector back in a dictionary
trans_state_dict = the_state._unpack_to_dict(trans_state_vec, do_transform=False, do_invtransform=False)
trans_delta_state_dict = the_state._unpack_to_dict(trans_delta_state_vec, do_transform=False, do_invtransform=False)
print 'transformed delta state ',trans_state_dict

# repeating the transformation again for just the pixel with an observation
transformed_param = []
for param, value in the_state.default_values.iteritems():
    if the_state.transformation_dict.has_key ( param ):
        transformed_param.append(the_state.transformation_dict[param](value))
    else:
        transformed_param.append(value)
print 'transformed parameters = ',transformed_param

# Get an emulator.
emu_fname = "/media/Data/emulators/70.0_55.0_67.5_115.0_prosail.npz"
#emu_fname = "/media/Data/emulators/PROSAIL/0030_0055_0030_prosail.npz"
gp = gp_emulator.MultivariateEmulator(dump=emu_fname)

# Forward model the reflectance from the input state. Returns reflectence per wavelength
# for 400 to 2500 nm (1 nm resolution).
print 'I expect my error text now'
specref, spechess, specderv = gp.predict(np.array(transformed_param).T, do_unc=True)
#, do_unc=True)

# choose band at 865 nm (NIR) only
spectrum = eoldas_observation_helpers.Spectral(np.array([865]),np.array([865])) 

# Create the observation image using the forward modelled value.
image = np.zeros_like(state_grid)
image[obslocation] = specref[spectrum.band_pass[0]]

transformed delta state  OrderedDict([('n', 1.6), ('cab', 0.81873075307798182), ('car', 0.99004983374916811), ('cbrown', 0.01), ('cw', 0.40656965974059917), ('cm', 0.049787068367863944), ('lai', array([[ 1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  0.22313016,  1.        ],
       [ 1.        ,  1.        ,  1.        ,  1.        ]])), ('ala', 0.7777777777777778), ('bsoil', 0.5), ('psoil', 0.9)])
transformed parameters =  [1.6, 0.81873075307798182, 0.99004983374916811, 0.01, 0.40656965974059917, 0.049787068367863944, 0.22313016014842982, 0.7777777777777778, 0.5, 0.9]
I expect my error text now
I can edit this


In [6]:
# We need to set and observation uncertainty.
band_uncertainty = np.array([0.02])

In [7]:
# Initialise the observation operator
the_observations = eoldas_ng.ObservationOperatorImageGP(state_grid,
                the_state, image[None,...], mask, gp,
                bu=band_uncertainty, factor=None,
                band_pass=spectrum.band_pass, bw=spectrum.bw, per_band=True)

#calculate the cost for the true state
cost, dercost = the_observations.der_cost(trans_state_dict, the_state.state_config)

print 'the cost is ', cost
# Calculate the cost and derivative for the shifted state
deltacost, deltadercost = the_observations.der_cost(trans_delta_state_dict, the_state.state_config)

After 5, the minimum cost was -1.132842e+03
the cost is  0.159526177944


In [8]:
# The cost function has been calculated in transformed space to we need epsilon in that space to
# calculate the finite difference.
epsilon_transformed = trans_delta_state_dict['lai'][obslocation] - trans_state_dict['lai'][obslocation]

fd_gradient = (deltacost - cost)/epsilon_transformed

print 'fd_gradient = ',fd_gradient
print'operator gradient = ', dercost.reshape(state_grid.shape)[obslocation]
print 'difference = ', (fd_gradient-dercost.reshape(state_grid.shape)[obslocation])

fd_gradient =  -3.64152792704
operator gradient =  -3.64058340828
difference =  -0.000944518764333


#Test 2.2 Hessian Code

This problem is set up in the same way as for the Jacobian test but we now call the der_der_cost method of the operator

In [9]:
Hess = the_observations.der_der_cost(trans_state_vec, the_state.state_config,
                                     the_state)
# Pull out the part of the Hessian that deals with LAI to inspect it.
print Hess.todense()[6].reshape(state_grid.shape)

[[  0.           0.           0.           0.        ]
 [  0.           0.          41.54129348   0.        ]
 [  0.           0.           0.           0.        ]]


In [10]:

print epsilon_transformed
# Finite difference using the gradients:
fd_hessian = (deltadercost-dercost)/epsilon_transformed
print fd_hessian.reshape(state_grid.shape)

-1.11565052197e-07
[[ -0.          -0.          -0.          -0.        ]
 [ -0.          -0.          32.12288711  -0.        ]
 [ -0.          -0.          -0.          -0.        ]]


In [12]:
print 'finite difference hessian = ', fd_hessian.reshape(state_grid.shape)[obslocation]
print 'operator hessian = ', Hess.todense()[6].reshape(state_grid.shape)[obslocation]

finite difference hessian =  32.1228871073
operator hessian =  41.5412934836
